In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv('Walmart.csv')
print(df.head())

   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1  05-02-2010    1643690.90             0        42.31       2.572   
1      1  12-02-2010    1641957.44             1        38.51       2.548   
2      1  19-02-2010    1611968.17             0        39.93       2.514   
3      1  26-02-2010    1409727.59             0        46.63       2.561   
4      1  05-03-2010    1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  


In [3]:
target_col = 'Weekly_Sales'
num_cols = [ 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
cat_cols = ['Store', 'Holiday_Flag']
feature_cols = cat_cols + num_cols
# может перевести номер филиала в категориальный признак ?

# что делать с DATE numerical? categorical?

# Divide on train and test

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split


In [5]:
one_hot_encoded = pd.get_dummies(df, columns=cat_cols)
df = one_hot_encoded
#df = pd.concat([df, one_hot_encoded], axis=1)
#df.drop(columns=cat_cols, axis=1, inplace= True)
#print(df)

In [6]:
#dataset = Dataset(X_tensor, y_tensor)

y = df[target_col]
X = df.drop(columns=target_col, axis=1)

In [7]:
#print(X)
#mean = df[num_cols].mean()
#std = df[num_cols].std()

# Standardize numerical features
#df[num_cols] = (df[num_cols] - mean) / std

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
mean = X_train[num_cols].mean()
std = X_train[num_cols].std()

# Standardize numerical features
X_train[num_cols] = (X_train[num_cols] - mean) / std
X_test[num_cols] = (X_test[num_cols] - mean) / std

In [10]:
#print(X_test)

In [11]:
X_train.drop(columns='Date', axis=1, inplace= True)
X_train = torch.tensor(X_train.values, dtype=torch.float32) 
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1) 

X_test.drop(columns='Date', axis=1, inplace= True)
X_test =  torch.tensor(X_test.values, dtype=torch.float32) 
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Sklearn LinearRegression 
проверка удалась, но теперь я не понимаю в чем проблема в нейронке

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

sklr = LinearRegression()
sklr.fit(X_train, y_train)

y_pred = sklr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Расчет R^2
r_squared = r2_score(y_test, y_pred)
print("R^2:", r_squared)

Mean Squared Error (MSE): 31261008000.0
R^2: 0.9041265019592664


# NN approach

In [33]:

class MyNNRegr(object):
    def __init__(self, model, optim, epochs = 3, loss = nn.MSELoss(), batch_size = 256):
        self.model = model
        self.optim = optim
        self.epochs = epochs
        self.loss = loss
        self.batch_size = batch_size

    def fit(self, X, y):
        print('1st ',X.shape, y.shape)
        
        
        num_samples = X.shape[0]

        for epoch in range(self.epochs):

            '''for i in range(num_samples):
                X_line = X[i]
                y_line = y[i]
                #print(X_line, y_line)
                train_pred = self.model(X_line)
                train_loss = self.loss(train_pred, y_line)
                print(train_pred, y_line)
                #print('loss: ', train_loss)
                #print('2nd', train_pred.shape, y_line.shape)

                self.optim.zero_grad()
                train_loss.backward()
                self.optim.step()'''
            for param in model.parameters():
                print(param)
            shuffled_indices = np.random.permutation(num_samples)
            
            for batch_start in range(0, num_samples, self.batch_size):
                batch_indices = shuffled_indices[batch_start:batch_start+self.batch_size]
                batch_X = X[batch_indices]
                batch_y = y[batch_indices]


                train_pred = self.model(batch_X)
                train_loss = self.loss(train_pred, batch_y)

                self.optim.zero_grad()
                train_loss.backward()
                self.optim.step()

    def predict(self, X):
        with torch.no_grad():
            pred = self.model(X)
        return pred


In [34]:

model = nn.Sequential(
    nn.Linear(51, 60),
    nn.ReLU(),
    nn.Linear(60, 1)
)

optimizer = optim.SGD(model.parameters(), lr=1e-4)

mynn = MyNNRegr(model, optimizer, epochs = 2)

In [35]:
from sklearn.metrics import mean_squared_error, r2_score

mynn.fit(X_train, y_train)
y_pred = mynn.predict(X_test)

print(y_pred[0:5])
print(y_test[0:5])

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Расчет R^2
r_squared = r2_score(y_test, y_pred)
print("R^2:", r_squared)

1st  torch.Size([5148, 51]) torch.Size([5148, 1])
Parameter containing:
tensor([[-0.0675, -0.0194,  0.0466,  ...,  0.0779,  0.1356,  0.1236],
        [ 0.0062, -0.0357, -0.0692,  ...,  0.1092,  0.1125, -0.1031],
        [-0.1206, -0.0438,  0.0997,  ...,  0.0125,  0.0305,  0.0632],
        ...,
        [ 0.1291,  0.1225,  0.0035,  ...,  0.1342, -0.0220,  0.0986],
        [ 0.0214,  0.0122,  0.0917,  ..., -0.0566,  0.0654,  0.1215],
        [-0.0464,  0.0950,  0.0010,  ...,  0.1152, -0.0255, -0.0798]],
       requires_grad=True)
Parameter containing:
tensor([-0.1149,  0.0339, -0.0298,  0.1140,  0.1247,  0.0428,  0.1081,  0.1295,
        -0.1083, -0.0053, -0.0654,  0.0921, -0.0279,  0.0051, -0.0661, -0.0590,
        -0.1392,  0.0021, -0.1036, -0.0697,  0.1214, -0.0772, -0.0061,  0.0238,
        -0.1068,  0.0370,  0.0535, -0.0491, -0.0964, -0.0306,  0.1063,  0.0231,
        -0.0034,  0.0827,  0.0292,  0.0260,  0.0217,  0.0171, -0.0878, -0.0171,
         0.1240, -0.0410, -0.0561,  0.1222,  

ValueError: Input contains NaN.